### Latent Variable Models

In [ ]:
import math
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=420, centers=3, n_features=2, random_state=40, cluster_std=1.6)

df = pd.DataFrame(dict(x = X[:, 0], y = X[:, 1], label=y))

colors = {0:'magenta', 1:'magenta', 2:'magenta'}
colors_1 = {0:'DeepSkyBlue', 1:'purple', 2:'yellow'}
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 5))
print (axes)
grouped = df.groupby('label')
for key, group in grouped:
  group.plot(ax=axes[0], kind='scatter', x='x', y='y', color=colors[key]) #label=key)
  group.plot(ax=axes[1], kind='scatter', x='x', y='y', color=colors_1[key], label=key)  

axes[1].set_title('Dataset with Labels', fontsize=12)
axes[0].set_title('Samples from Make_Blob Dataset', fontsize=12)
plt.tight_layout()
# plt.savefig('/content/gdrive/My Drive/Make_Blobs.png', dpi=200)
plt.show()


In [ ]:
### Use the same data-set and Use K-Means 
from sklearn.cluster import KMeans
X, y = make_blobs(n_samples=420, centers=3, n_features=2, random_state=40, cluster_std=1.6)# same as before

kmeans = KMeans(3, random_state=0)
lab_K = kmeans.fit(X).predict(X)

fig = plt.figure(figsize=(7, 7))
plt.scatter(X[:, 0], X[:, 1], c=lab_K, s=40, cmap='viridis')
plt.show()

In [ ]:
### Use the Same Make Blobs Model  But Now Gaussian Mix Model 
from sklearn.mixture import GaussianMixture
X, y = make_blobs(n_samples=420, centers=3, n_features=2, random_state=40, cluster_std=1.6)# same as before
# X = X[:, ::-1]
gmm = GaussianMixture(n_components=3).fit(X)
labels = gmm.predict(X)

probabs = gmm.predict_proba(X)
sizes = 50* (probabs.max(1)**2)


fig = plt.figure(figsize=(9, 5))
plt.subplot(1, 2, 1)
plt.scatter(X[:, 0], X[:, 1], c=lab_K, s=40, cmap='viridis')
plt.title("Hard Clustering with K-Means")

plt.subplot(1, 2, 2)
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', s=sizes)
plt.title("Soft Clustering with GMM")
# plt.savefig('/content/gdrive/My Drive/KMeans_GMM.png', dpi=200)
plt.show()

### Use the EM Steps for GMM
Ref : https://github.com/suvoooo/Machine_Learning/blob/eb3d96fbf774a428a1e0124afaf29f166b3c19d8/ExMax_ALgo/LVM.ipynb

In [1]:
import math
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
# Load the samples using numpy load.
samples_folder = np.load('samples.npz')
# samples_folder = np.load('/dataset/iris.csv')

print ("samples folder: ", samples_folder)
data = samples_folder['data']
print ("data shape: ", data.shape)
# print ("data : ", data)
data_0 = data[:, 0]
data_1 = data[:, 1]
# print ("data_0 : ", data_0)
# print ("data_1 : ", data_1)
fig = plt.figure(figsize=(6, 4))
plt.plot(data_0, data_1, linestyle='None', marker='o', markersize=5, color='navy')
plt.xlabel('data[:. 0]', fontsize=12)
plt.ylabel('data[:, 1]', fontsize=12)
plt.show()

In C:\Users\Jaha\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Jaha\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Jaha\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Jaha\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Jaha\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle

samples folder:  <numpy.lib.npyio.NpzFile object at 0x0000022F9683C208>
data shape:  (280, 2)


<Figure size 600x400 with 1 Axes>

In [2]:

pi0 = samples_folder['pi0']
mu0 = samples_folder['mu0']
sigma0 = samples_folder['sigma0']

print ("check the shape of pi0, mu0, sigma0 :", pi0.shape, mu0.shape, sigma0.shape) # possibly 3 clusters 
print ("check some values of pi0: ", pi0)
print ("check some values of mu0: ", '\n', mu0, '\n')
print ("check some values of sigma0: ", '\n', sigma0)

check the shape of pi0, mu0, sigma0 : (3,) (3, 2) (3, 2, 2)
check some values of pi0:  [0.3451814  0.6066179  0.04820071]
check some values of mu0:  
 [[-0.71336192  0.90635089]
 [ 0.76623673  0.82605407]
 [-1.32368279 -1.75244452]] 

check some values of sigma0:  
 [[[ 1.00490413  1.89980228]
  [ 1.89980228  4.18354574]]

 [[ 1.96867815  0.78415336]
  [ 0.78415336  1.83319942]]

 [[ 0.19316335 -0.11648642]
  [-0.11648642  1.98395967]]]


In [5]:
#### E step for GMM 
### Use multivariate normal for Gaussian distribution 
from scipy.stats import multivariate_normal

def E_step(data, pi, mu, sigma):
    N = data.shape[0] # number of data-points
    K = pi.shape[0] # number of clusters, following notation used before in description  
    d = mu.shape[1] # dimension of each data point, think of these as attributes
    gamma = np.zeros((N, K)) # this is basically responsibility which should be equal to posterior. 

    for nk in range(K):
        gamma[:, nk] = pi[nk] * multivariate_normal.pdf(data, mean=mu[nk], cov=sigma[nk]) 
    # calculate responsibility for each cluster
    gamma = gamma/np.sum(gamma, axis=1, keepdims=True) 
    # use the sum over all the clusters, thus axis=1. Denominator term. 
    # print ("gamma shape: ", gamma.shape)
    return gamma

In [ ]:
#Remove Outliers from Data for every clusters...(How???)



In [6]:
def M_step(data, gamma):
    N, D = data.shape 
    K = gamma.shape[1] # use the posterior shape calculated in E-step to determine the no. of clusters  
    pi = np.zeros(K)
    mu = np.zeros((K, D))
#     print("MU : ", mu)
    sigma = np.zeros((K, D, D)) 

    for ik in range(K):
        n_k = gamma[:, ik].sum() # we use the definintion of N_k 
        pi[ik] = n_k/N # definition of the weights
        elements = np.reshape(gamma[:, ik], (gamma.shape[0], 1)) 
        # get each columns and reshape it (K, 1) form so that later broadcasting is possible. 
        mu[ik,:] = (np.multiply( elements,  data)).sum(axis=0) / n_k  
        sigma_sum = 0.
        for i in range(N):
            var = data[i] - mu[ik]
            sigma_sum = sigma_sum + gamma[i, ik] * np.outer(var, var)# outer product creates the covariance matrix
        sigma[ik,:] = sigma_sum/n_k   
    return pi, mu, sigma

In [ ]:
# gamma = E_step(data, pi, mu, sigma)
# pi, mu, sigma = M_step(data, gamma)

In [7]:
def elbo(data, gamma, pi, mu, sigma):
    N = data.shape[0] # no. of data-points
    K = gamma.shape[1] # no. of clusters
    d = data.shape[1] # dim. of each object

    loss = 0.
    for i in range(N):
        x = data[i]
        for k in range(K):
            pos_dist = gamma[i, k] ## p(z_i=k|x) = gamma_ik
            log_lik = np.log(multivariate_normal.pdf(x, mean=mu[k, :], cov=sigma[k, :, :]) + 1e-20) # log p(x|z)
            log_q = np.log(gamma[i, k] + 1e-20) # log q(z) = log p(z_i=k|x)
            log_pz = np.log(pi[k] + 1e-20)  # log p(z_k =1) =\pi _k
            loss = (loss + np.multiply(pos_dist, log_pz) + np.multiply(pos_dist, log_lik) +  
            np.multiply(pos_dist, -log_q) )
  #print ("check loss: ", loss)

    return loss

In [22]:
def train_loop(data, K, tolerance=1e-3, max_iter=50, restart=10):
    N, d = data.shape
    elbo_best = -np.inf # loss set to the lowest value 
    pi_best = None
    mu_best = None
    sigma_best = None
    gamma_f = None
    for _ in range(restart):
        pi = np.ones(K) / K # if 3 clusters then an array of [.33, .33, .33] # the sum of pi's should be one 
        # that's why normalized  
        mu = np.random.rand(K, d) # no condition on 
        sigma = np.tile(np.eye(d), (K, 1, 1)) # to initialize sigma we first start with ones only at the diagonals
        # the sigmas are postive semi-definite and symmetric  
        last_iter_loss = None
        all_losses = []
        try:

            for i in range(max_iter):
                gamma = E_step(data, pi, mu, sigma)
                pi, mu, sigma = M_step(data, gamma)
                loss = elbo(data, gamma, pi, mu, sigma)
                print("data_shape : ", data.shape)
                print("gamma :", gamma)
                print(gamma.shape)
                print("mu :", mu)
                print(mu.shape)
                print("pi :", pi)
                print(pi.shape)
                print("sigma :", sigma)
                print(sigma.shape)
                
                if loss > elbo_best:
                    elbo_best = loss
                    pi_best = pi 
                    mu_best = mu
                    sigma_best = sigma
                    gamma_f = gamma
                if last_iter_loss and abs((loss-last_iter_loss)/last_iter_loss) < tolerance: # insignificant improvement
                    break 
                last_iter_loss = loss
                all_losses.append(loss)
        except np.linalg.LinAlgError: # avoid the delta function situation 
            pass 
    gamma_df = pd.DataFrame(gamma_f)
    gamma_df.to_excel("result/gamma_f.xlsx")
    return elbo_best, pi_best, mu_best, sigma_best, all_losses, gamma_f

In [21]:
best_loss, pi_best, mu_best, sigma_best, ls_lst, final_posterior = train_loop(data, 3)

data_shape :  (280, 2)
gamma : [[0.43375064 0.23202008 0.33422927]
 [0.41966349 0.24766933 0.33266717]
 [0.42636996 0.23515295 0.33847709]
 [0.48408011 0.17415162 0.34176827]
 [0.48476345 0.16199603 0.35324052]
 [0.43140573 0.18690368 0.38169058]
 [0.47419306 0.06430218 0.46150476]
 [0.39225889 0.31790071 0.2898404 ]
 [0.42285425 0.26110144 0.3160443 ]
 [0.26731216 0.4960096  0.23667824]
 [0.30028039 0.44455826 0.25516135]
 [0.43268619 0.23073959 0.33657422]
 [0.30491069 0.41188525 0.28320406]
 [0.4874588  0.12462981 0.3879114 ]
 [0.45097427 0.16359509 0.38543064]
 [0.3554641  0.34868841 0.29584749]
 [0.35332841 0.28246921 0.36420238]
 [0.47078013 0.19126583 0.33795404]
 [0.30993528 0.40459663 0.28546809]
 [0.4864691  0.12069302 0.39283788]
 [0.38060244 0.31776676 0.3016308 ]
 [0.48500135 0.05993632 0.45506233]
 [0.43682887 0.24999808 0.31317304]
 [0.39669713 0.30738715 0.29591572]
 [0.42037685 0.22484263 0.35478053]
 [0.41925409 0.26036446 0.32038145]
 [0.31652823 0.38419229 0.2992794

gamma : [[3.36322669e-01 4.32820363e-01 2.30856968e-01]
 [3.25601224e-01 4.44280068e-01 2.30118708e-01]
 [3.34565730e-01 4.28675393e-01 2.36758877e-01]
 [3.79156366e-01 3.70756305e-01 2.50087330e-01]
 [4.17224279e-01 2.98827386e-01 2.83948336e-01]
 [4.17501708e-01 2.45097657e-01 3.37400635e-01]
 [4.80908763e-01 9.66708778e-05 5.18994566e-01]
 [3.55040782e-01 4.09023267e-01 2.35935950e-01]
 [3.26879376e-01 4.55505737e-01 2.17614887e-01]
 [3.82399621e-01 2.37450055e-01 3.80150324e-01]
 [3.73893828e-01 2.99911120e-01 3.26195052e-01]
 [3.37744866e-01 4.28204378e-01 2.34050756e-01]
 [2.61497820e-01 4.79719805e-01 2.58782375e-01]
 [5.21344059e-01 8.19482676e-02 3.96707674e-01]
 [4.62258111e-01 1.76435977e-01 3.61305912e-01]
 [3.11371801e-01 4.45420518e-01 2.43207682e-01]
 [2.29998343e-01 5.22433296e-01 2.47568362e-01]
 [3.61647412e-01 4.00641544e-01 2.37711044e-01]
 [2.64767354e-01 4.79216582e-01 2.56016063e-01]
 [5.27471667e-01 6.37885787e-02 4.08739755e-01]
 [3.23265046e-01 4.44174576e-01 

data_shape :  (280, 2)
gamma : [[1.97630277e-01 7.25727643e-01 7.66420797e-02]
 [1.94286546e-01 7.22139442e-01 8.35740128e-02]
 [2.01776505e-01 7.10129449e-01 8.80940459e-02]
 [2.43764612e-01 6.86775302e-01 6.94600858e-02]
 [3.23677669e-01 5.60417923e-01 1.15904408e-01]
 [3.80639932e-01 3.63111828e-01 2.56248239e-01]
 [2.26752840e-01 2.17367863e-07 7.73246943e-01]
 [2.94014314e-01 6.09206752e-01 9.67789342e-02]
 [1.96528871e-01 7.38032084e-01 6.54390448e-02]
 [4.53268412e-01 8.59187353e-02 4.60812853e-01]
 [4.50956113e-01 2.07177643e-01 3.41866244e-01]
 [2.00713995e-01 7.18153666e-01 8.11323386e-02]
 [2.72976372e-01 5.06830942e-01 2.20192686e-01]
 [5.44408816e-01 7.40213955e-02 3.81569788e-01]
 [4.53126216e-01 2.38487763e-01 3.08386021e-01]
 [2.60836280e-01 5.95480298e-01 1.43683422e-01]
 [1.79284740e-01 6.97527791e-01 1.23187469e-01]
 [2.15939231e-01 7.20218782e-01 6.38419872e-02]
 [2.68164047e-01 5.22225039e-01 2.09610914e-01]
 [5.38272411e-01 4.63823028e-02 4.15345287e-01]
 [2.42161

data_shape :  (280, 2)
gamma : [[1.89591832e-01 7.75018152e-01 3.53900158e-02]
 [1.90961088e-01 7.64136127e-01 4.49027843e-02]
 [1.94628563e-01 7.57602323e-01 4.77691136e-02]
 [2.20547915e-01 7.61248257e-01 1.82038286e-02]
 [2.86476463e-01 6.72275310e-01 4.12482274e-02]
 [3.47670499e-01 4.28537389e-01 2.23792111e-01]
 [4.69593471e-02 3.57355093e-07 9.53040296e-01]
 [3.05718126e-01 6.58174654e-01 3.61072198e-02]
 [1.96071001e-01 7.78823493e-01 2.51055063e-02]
 [4.89703120e-01 8.51409014e-02 4.25155979e-01]
 [4.99408873e-01 2.18807365e-01 2.81783762e-01]
 [1.92104035e-01 7.68343662e-01 3.95523030e-02]
 [3.39104698e-01 4.65232666e-01 1.95662636e-01]
 [5.09655926e-01 1.39160442e-01 3.51183632e-01]
 [4.03883991e-01 3.21267355e-01 2.74848654e-01]
 [2.86303036e-01 6.13911148e-01 9.97858160e-02]
 [2.51051789e-01 6.46519287e-01 1.02428923e-01]
 [1.98710071e-01 7.82645941e-01 1.86439884e-02]
 [3.29685826e-01 4.85508346e-01 1.84805828e-01]
 [4.89469935e-01 8.98833858e-02 4.20646679e-01]
 [2.55166

data_shape :  (280, 2)
gamma : [[1.94855794e-01 7.92904345e-01 1.22398606e-02]
 [2.02296972e-01 7.78030579e-01 1.96724485e-02]
 [2.02254146e-01 7.76440862e-01 2.13049918e-02]
 [2.01759654e-01 7.95609964e-01 2.63038204e-03]
 [2.49724838e-01 7.41679389e-01 8.59577305e-03]
 [3.28772406e-01 4.77475924e-01 1.93751670e-01]
 [1.85311754e-02 1.50120363e-06 9.81467323e-01]
 [3.22176651e-01 6.70402002e-01 7.42134655e-03]
 [2.07305686e-01 7.86387048e-01 6.30726593e-03]
 [5.67048065e-01 1.01823940e-01 3.31127994e-01]
 [5.72276956e-01 2.48928574e-01 1.78794470e-01]
 [1.97284197e-01 7.87823254e-01 1.48925496e-02]
 [4.02748350e-01 4.16959434e-01 1.80292216e-01]
 [5.02392744e-01 2.69467133e-01 2.28140123e-01]
 [3.71540290e-01 4.12402366e-01 2.16057344e-01]
 [3.25631273e-01 6.16377823e-01 5.79909035e-02]
 [3.32386082e-01 5.50373446e-01 1.17240472e-01]
 [1.89278045e-01 8.07500269e-01 3.22168606e-03]
 [3.91729227e-01 4.40705990e-01 1.67564783e-01]
 [4.91707962e-01 1.89944326e-01 3.18347712e-01]
 [2.81263

...

data_shape :  (280, 2)
gamma : [[4.64950663e-01 5.35049103e-01 2.33694750e-07]
 [3.87340870e-01 6.12650188e-01 8.94169680e-06]
 [4.20202438e-01 5.79791882e-01 5.68000233e-06]
 [3.06188385e-01 6.93811615e-01 1.44078376e-13]
 [5.09977268e-01 4.90022732e-01 3.28197578e-12]
 [5.23041357e-01 4.61770100e-01 1.51885433e-02]
 [9.99892576e-01 3.42702307e-08 1.07390141e-04]
 [6.23548501e-01 3.76451245e-01 2.53401268e-07]
 [4.75665966e-01 5.24333988e-01 4.54678940e-08]
 [3.34099471e-04 8.40029798e-02 9.15662921e-01]
 [2.05190885e-02 4.80503508e-01 4.98977403e-01]
 [4.60890748e-01 5.39108669e-01 5.83172790e-07]
 [4.43013381e-05 1.67792403e-01 8.32163295e-01]
 [9.69066133e-01 3.09338647e-02 2.55610277e-09]
 [8.46823917e-01 1.53018520e-01 1.57562866e-04]
 [9.40973801e-02 8.86291088e-01 1.96115321e-02]
 [8.64406042e-07 3.06710161e-02 9.69328119e-01]
 [3.79478884e-01 6.20521116e-01 4.94032855e-12]
 [1.00280859e-04 2.04788371e-01 7.95111349e-01]
 [9.82634960e-01 1.73650318e-02 7.88295818e-09]
 [3.20221

data_shape :  (280, 2)
gamma : [[2.85445168e-01 7.14554307e-01 5.25648033e-07]
 [2.45826413e-01 7.54157360e-01 1.62271590e-05]
 [2.69263949e-01 7.30725394e-01 1.06573050e-05]
 [1.60425212e-01 8.39574788e-01 7.19243045e-13]
 [3.11476222e-01 6.88523778e-01 1.49186980e-11]
 [5.16146418e-01 4.58399863e-01 2.54537189e-02]
 [9.99860816e-01 4.30215114e-09 1.39179284e-04]
 [4.68455510e-01 5.31543743e-01 7.47050834e-07]
 [2.92888720e-01 7.07111161e-01 1.18893589e-07]
 [3.67291793e-04 3.01497416e-02 9.69482967e-01]
 [2.18678273e-02 3.16437219e-01 6.61694954e-01]
 [2.87168212e-01 7.12830542e-01 1.24562638e-06]
 [5.19408958e-05 6.57032663e-02 9.34244793e-01]
 [9.50341897e-01 4.96580926e-02 1.01732487e-08]
 [8.11629317e-01 1.88041302e-01 3.29381128e-04]
 [8.35457839e-02 8.87288757e-01 2.91654591e-02]
 [1.00442724e-06 4.97883722e-03 9.95020158e-01]
 [2.04283805e-01 7.95716195e-01 2.02135718e-11]
 [1.15698513e-04 8.95734290e-02 9.10310873e-01]
 [9.74270228e-01 2.57297438e-02 2.86981937e-08]
 [2.26397

data_shape :  (280, 2)
gamma : [[1.65511951e-01 8.34486540e-01 1.50890422e-06]
 [1.60148669e-01 8.39813043e-01 3.82882270e-05]
 [1.73356891e-01 8.26617737e-01 2.53725410e-05]
 [9.36368231e-02 9.06363177e-01 5.83167663e-12]
 [1.81966502e-01 8.18033497e-01 8.94297018e-11]
 [5.23719640e-01 4.33554250e-01 4.27261104e-02]
 [9.99850223e-01 5.95574622e-09 1.49770982e-04]
 [2.99740004e-01 7.00257159e-01 2.83668948e-06]
 [1.63902157e-01 8.36097431e-01 4.11786585e-07]
 [6.18607627e-04 5.84439978e-03 9.93536993e-01]
 [2.66753745e-02 1.30209927e-01 8.43114699e-01]
 [1.71111606e-01 8.28885028e-01 3.36646667e-06]
 [9.80168743e-05 1.00057315e-02 9.89896252e-01]
 [8.94659039e-01 1.05340916e-01 4.53743034e-08]
 [7.44890552e-01 2.54404698e-01 7.04749957e-04]
 [1.04865518e-01 8.32101413e-01 6.30330690e-02]
 [2.41628029e-06 3.12003115e-04 9.99685581e-01]
 [1.11184036e-01 8.88815964e-01 1.23278583e-10]
 [2.03502514e-04 1.60166206e-02 9.83779877e-01]
 [9.44126732e-01 5.58731512e-02 1.16381823e-07]
 [1.74613

data_shape :  (280, 2)
gamma : [[8.25546415e-02 9.17435045e-01 1.03137178e-05]
 [1.04687985e-01 8.95119741e-01 1.92274061e-04]
 [1.08642852e-01 8.91229129e-01 1.28019290e-04]
 [5.06376707e-02 9.49362329e-01 3.13858056e-10]
 [8.60594072e-02 9.13940590e-01 2.44557360e-09]
 [5.67023098e-01 3.40195951e-01 9.27809505e-02]
 [9.99802466e-01 1.05460915e-07 1.97428771e-04]
 [1.34393769e-01 8.65583172e-01 2.30588924e-05]
 [7.26069277e-02 9.27389399e-01 3.67340117e-06]
 [1.85751687e-03 2.85473232e-04 9.97857010e-01]
 [3.60546933e-02 1.89973216e-02 9.44947985e-01]
 [9.02669589e-02 9.09712264e-01 2.07769655e-05]
 [3.41574318e-04 2.31832404e-04 9.99426593e-01]
 [6.77628280e-01 3.22371202e-01 5.17619355e-07]
 [6.27282708e-01 3.70456533e-01 2.26075898e-03]
 [1.91546598e-01 5.81497479e-01 2.26955923e-01]
 [1.31351327e-05 1.87339871e-06 9.99984991e-01]
 [5.11736199e-02 9.48826376e-01 3.67310331e-09]
 [6.13927752e-04 4.87209977e-04 9.98898862e-01]
 [7.95559112e-01 2.04439700e-01 1.18845210e-06]
 [1.51582

gamma : [[4.74810372e-02 9.52493328e-01 2.56343946e-05]
 [7.82744531e-02 9.21305169e-01 4.20378157e-04]
 [7.92630926e-02 9.20454635e-01 2.82272052e-04]
 [2.06540261e-02 9.79345972e-01 2.18726250e-09]
 [3.53853723e-02 9.64614616e-01 1.17836332e-08]
 [6.19471308e-01 2.51982821e-01 1.28545871e-01]
 [9.99750733e-01 9.49482813e-07 2.48317271e-04]
 [5.82446426e-02 9.41702592e-01 5.27656411e-05]
 [3.48200471e-02 9.65170047e-01 9.90543641e-06]
 [4.85080304e-03 8.46132270e-05 9.95064584e-01]
 [5.27720531e-02 8.85341335e-03 9.38374534e-01]
 [5.54200707e-02 9.44530383e-01 4.95466304e-05]
 [9.61293254e-04 3.17933442e-05 9.99006913e-01]
 [4.12116634e-01 5.87881850e-01 1.51621110e-06]
 [5.67513401e-01 4.28555427e-01 3.93117165e-03]
 [2.42658729e-01 4.16348277e-01 3.40992994e-01]
 [4.31672797e-05 7.75754273e-08 9.99956755e-01]
 [2.08719359e-02 9.79128045e-01 1.89096867e-08]
 [1.57675876e-03 7.73103761e-05 9.98345931e-01]
 [5.59635184e-01 4.40361310e-01 3.50598827e-06]
 [1.29390943e-01 8.64448460e-01 

In [ ]:
print ("starting loss, best_loss: ", ls_lst[0], ',',  best_loss)
print ("best pi", pi_best) 
print ("best mu: ", mu_best)

print ("best sigma: ", sigma_best)

In [ ]:

print("what about final posterior; ", final_posterior.shape)
print ("check one of the values: ", final_posterior[5, :])

In [ ]:
fig = plt.figure(figsize=(6,5))
plt.plot(range(len(ls_lst)), ls_lst, linestyle='--', color='orange', label='ELBO')
plt.xlabel('Steps', fontsize=11)
plt.ylabel('ELBO', fontsize=11)
plt.legend(fontsize=12)

### Implement using Scikit Learn

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=3, n_init=10)
gmm.fit(data)

In [ ]:
print ("using sklearn")
print ("best pi : ", gmm.weights_)

print ("best mu :", gmm.means_)

### Plot Contours

In [ ]:
# def plot_densities(data, mu, sigma, alpha = 0.5, colors='grey'):
    # grid_x, grid_y = np.mgrid[X[:,0].min():X[:,0].max():200j,
                    #  X[:,1].min():X[:,1].max():200j]
    # grid = np.stack([grid_x, grid_y], axis=-1)
# 
    # for mu_c, sigma_c in zip(mu, sigma):
        # plt.contour(grid_x, grid_y, multivariate_normal(mu_c, sigma_c).pdf(grid), colors=colors, alpha=alpha)



def plot_contours(data, means, covs):
    """visualize the gaussian components over the data"""
    plt.figure()
    plt.plot(data[:, 0], data[:, 1], 'ko')

    delta = 0.025
    k = means.shape[0]
    x = np.arange(-2.5, 10.0, delta)
    y = np.arange(-2.5, 10.0, delta)
    x_grid, y_grid = np.meshgrid(x, y)
    coordinates = np.array([x_grid.ravel(), y_grid.ravel()]).T

    col = ['green', 'red', 'indigo']
    for i in range(k):
        mean = means[i]
        cov = covs[i]
        z_grid = multivariate_normal(mean, cov).pdf(coordinates).reshape(x_grid.shape)
        plt.contour(x_grid, y_grid, z_grid, colors = col[i])
    plt.tight_layout()

In [ ]:
print ("check whether the best one converged: ", gmm.converged_)
print ("how many steps to convergence: ", gmm.n_iter_)

In [ ]:
fig= plt.figure(figsize=(8, 6))
plot_contours(data, gmm.means_, gmm.covariances_)
plt.xlabel("data[:, 0]", fontsize=12)
plt.ylabel("data[:, 1]", fontsize=12)
plt.show()